In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("../input/titanic/train.csv",sep = ',')
print(data)

In [ ]:
data1 = data.loc[:,['Pclass','Sex','Age','SibSp','Parch']]
print(data1)

In [ ]:
for s in range(0,len(data1)):
    if data1['Sex'].iloc[s] == 'male':
        data1['Sex'].iloc[s]  = 1
    else:
        data1['Sex'].iloc[s]  = 0
print(data1)

In [ ]:
age_v = data1['Age']
t = age_v.isnull()
for s in range(0,len(data1)):
    if t.iloc[s] == True :
        data1['Age'].iloc[s] = 0.4
    else:
        data1['Age'].iloc[s] /=  50
print(data1)

In [ ]:
data = np.array(data1,dtype=np.float32)
import torch
data = torch.Tensor(data)

In [ ]:
print(data)

In [ ]:
data1 = pd.read_csv("../input/titanic/train.csv",sep = ',')
data_y = data1.loc[:,['Survived']]
print(data_y)
data_y = np.array(data_y,dtype=np.float32)
data_y = torch.tensor(data_y)

In [ ]:
print(data_y)

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

class diabetes(Dataset):
    def __init__(self):
        self.len = len(data)
        self.x = data
        self.y = data_y
    def __getitem__(self,index):
        return self.x[index],self.y[index]
    def __len__(self):
        return self.len
dataset = diabetes()
train_data = DataLoader(dataset = dataset,batch_size=100,shuffle=True)

In [ ]:

class CEB(torch.nn.Module):
    def __init__(self):
        super(CEB,self).__init__()
        self.linear1 = torch.nn.Linear(5,10)
        self.linear2 = torch.nn.Linear(10,6)
        self.linear3 = torch.nn.Linear(6,4)
        self.linear4 = torch.nn.Linear(4,1)
        self.sigmoid = torch.nn.Sigmoid()
    def forward(self,x):
        x = self.sigmoid(self.linear1(x))
        x = self.sigmoid(self.linear2(x))
        x = self.sigmoid(self.linear3(x))
        x = self.sigmoid(self.linear4(x))
        return x
model = CEB()
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.1)
epochs = 1000
for epoch in range(0,epochs):
    for i,data in enumerate(train_data,0):
        inputs ,labels = data
        y_pred = model(inputs)
        loss = criterion(y_pred,labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(epoch,loss.item())    

In [ ]:
test = pd.read_csv('../input/titanic/test.csv',sep=',')
test = test.loc[:,['Pclass','Sex','Age','SibSp','Parch']]
for s in range(0,len(test)):
    if test['Sex'].iloc[s] == 'male':
        test['Sex'].iloc[s]  = 1
    else:
        test['Sex'].iloc[s]  = 0
age_v = test['Age']
t = age_v.isnull()
for s in range(0,len(test)):
    if t.iloc[s] == True :
        test['Age'].iloc[s] = 0.4
    else:
        test['Age'].iloc[s] /=  50
print(test)


In [ ]:
test = np.array(test,dtype=np.float32)
test = torch.tensor(test)
y = model(test)
print(y)

In [ ]:
for s in range(0,len(test)):
    if y[s]>=0.5:
        y[s]=1
    else:
        y[s]=0
print(y)

In [ ]:
c = y.detach().numpy()
y = np.array(c,dtype=np.uint8)

In [ ]:
print(y)

In [ ]:
y_index = np.linspace(892,1309,418,dtype=np.uint32)
y_index = y_index.reshape(418,1)
print(y_index)

In [ ]:
ans = np.hstack((y_index,y))

In [ ]:
print(ans)

In [ ]:
np.savetxt('123.csv',ans,delimiter=',')